因为上传文件以「表单数据」形式发送。

所以接收上传文件，要预先安装 python-multipart。

例如： pip install python-multip

定义文件参数时使用 UploadFile：art。

In [1]:
import uvicorn
from fastapi import FastAPI, UploadFile
app = FastAPI()

@app.post("/uploadfile/")
async def create_upload_file(file: UploadFile):
    return {"filename": file.filename}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [23576]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:63751 - "POST /uploadfile/ HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:63759 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:63759 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:63806 - "POST /uploadfile/ HTTP/1.1" 400 Bad Request


Did not find boundary character 57 at index 4


INFO:     127.0.0.1:64006 - "POST /uploadfile/ HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:64026 - "POST /uploadfile/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:64070 - "POST /uploadfile/ HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:64125 - "POST /uploadfile/ HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:64229 - "POST /uploadfile/ HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:51473 - "POST /uploadfile/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:51492 - "POST /uploadfile/ HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:51501 - "POST /uploadfile/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23576]


In [ ]:
url = "http://127.0.0.1:8009/uploadfile/"
files = {"file": open('19.JPG', "rb")}
res = requests.post(url, files=files)
res.text

# 先把一张图片放在和ipynb文件路径相同的地方，命名为 19.JPG
# 在另一个ipynb文件中运行代码，会得到 '{"filename":"19.JPG"}'

UploadFile 的属性如下：

filename：上传文件名字符串（str），例如， myimage.jpg；
content_type：内容类型（MIME 类型 / 媒体类型）字符串（str），例如，image/jpeg；
file： SpooledTemporaryFile（ file-like 对象）。其实就是 Python文件，可直接传递给其他预期 file-like 对象的函数或

UploadFile 支持以下 async 方法，（使用内部 SpooledTemporaryFile）可调用相应的文件方法。

write(data)：把 data （str 或 bytes）写入文件；
read(size)：按指定数量的字节或字符（size (int)）读取文件内容；
seek(offset)：移动至文件 offset （int）字节处的位置；
例如，await myfile.seek(0) 移动到文件开头；
执行 await myfile.read() 后，需再次读取已读取内容时，这种方法特别好用；
close()：关闭文件。
因为上述方法都是 async 方法，要搭配「a
使用 async 方法时，FastAPI 在线程池中执行文件方法，并 await 操作完成。

例如，在 async 路径操作函数 内，要用以下方式读取文件内容：
contents = await myfile.read()

在普通 def 路径操作函数 内，则可以直接访问 UploadFile.file，例如：
contents = myfile.file.read()

wait」使用。支持库。

## 可选文件上传

您可以通过使用标准类型注解并将 None 作为默认值的方式将一个文件参数设为可选:

In [ ]:
import uvicorn
from fastapi import FastAPI, UploadFile
app = FastAPI()

@app.post("/uploadfile/")
async def create_upload_file(file: UploadFile | None = None):
    if not file:
        return {"message": "No upload file sent"}
    else:
        return {"filename": file.filename}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

## 多文件上传

FastAPI 支持同时上传多个文件。

可用同一个「表单字段」发送含多个文件的「表单数据」。

上传多个文件时，要声明含 bytes 或 UploadFile 的列表（List）：

In [2]:
import uvicorn
from fastapi import FastAPI, UploadFile
app = FastAPI()

@app.post("/uploadfiles/")
async def create_upload_files(files: list[UploadFile]):
    return {"filenames": [file.filename for file in files]}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [23576]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:51826 - "POST /uploadfiles/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23576]


In [ ]:
url = "http://127.0.0.1:8009/uploadfiles/"
files = [("files", open('19.JPG', "rb")),("files", open('89.JPG', "rb"))]
res = requests.post(url, files=files)
res.text

# 先把两张图片放在和ipynb文件路径相同的地方，命名为 19.JPG 和 89.JPG
# 在另一个ipynb文件中运行代码，会得到 '{"filenames":["19.JPG","89.JPG"]}'

## 带前端测试页面

In [ ]:
import uvicorn
from fastapi import FastAPI, UploadFile
from fastapi.responses import HTMLResponse
app = FastAPI()

@app.post("/uploadfiles/")
async def create_upload_files(files: list[UploadFile]):
    return {"filenames": [file.filename for file in files]}

@app.get("/")
async def main():
    content = """
<body>
<form action="/uploadfiles/" enctype="multipart/form-data" method="post">
<input name="files" type="file" multiple>
<input type="submit">
</form>
</body>
    """
    return HTMLResponse(content=content)

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [23576]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:51896 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:51895 - "POST /uploadfiles/ HTTP/1.1" 200 OK


打开浏览器 http://127.0.0.1:8009/ 可以上传文件